# Multilayer Perception (MLP)

- ### Step 1: Import Necessary Libraries
  ##### We import all the necessary libraries for data manipulation, model building, and hyperparameter tuning.

In [14]:
# Import necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import kerastuner as kt
from tensorflow import keras


- ### Step 2: Load and Preprocess Data
  ##### We load the dataset, preprocess the RGB columns, encode the labels, split the data into training and testing sets, and standardize the features.

In [9]:
#Initial form of the data 
data = pd.read_csv('demmo_data.csv')

data.head()

,Emotional_Word,RGB_1,RGB_2,RGB_3
0,Cute,"[251, 167, 157]","[255, 242, 124]","[179, 22, 61]"
1,Childlike,"[251, 103, 89]","[255, 242, 124]","[153, 216, 212]"
2,Pretty,"[251, 167, 157]","[255, 242, 63]","[78, 181, 135]"
3,Sweet,"[251, 103, 89]","[253, 192, 145]","[251, 174, 193]"
4,Amusing,"[253, 166, 74]","[140, 201, 25]","[90, 177, 132]"


In [10]:
# Preprocess RGB columns
data['RGB_1'] = data['RGB_1'].apply(lambda x: eval(x))
data['RGB_2'] = data['RGB_2'].apply(lambda x: eval(x))
data['RGB_3'] = data['RGB_3'].apply(lambda x: eval(x))

# Extract features and labels
X = np.hstack([np.vstack(data['RGB_1']), np.vstack(data['RGB_2']), np.vstack(data['RGB_3'])])
y = data['Emotional_Word']

data.head()

,Emotional_Word,RGB_1,RGB_2,RGB_3
0,Cute,"[251, 167, 157]","[255, 242, 124]","[179, 22, 61]"
1,Childlike,"[251, 103, 89]","[255, 242, 124]","[153, 216, 212]"
2,Pretty,"[251, 167, 157]","[255, 242, 63]","[78, 181, 135]"
3,Sweet,"[251, 103, 89]","[253, 192, 145]","[251, 174, 193]"
4,Amusing,"[253, 166, 74]","[140, 201, 25]","[90, 177, 132]"


- ### Step 3: Encode Labels and Split Data
  ##### Encode the labels, split the data into training and testing sets, and standardize the features.

In [11]:
# Extract features and labels
X = np.hstack([np.vstack(data['RGB_1']), np.vstack(data['RGB_2']), np.vstack(data['RGB_3'])])
y = data['Emotional_Word']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.2, random_state=42)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


- ### Step 4: Define the Model Building Function for Keras Tuner
  ##### This function defines the MLP model and includes hyperparameters for tuning the number of units, dropout rate, number of layers, and learning rate.

In [15]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu', input_shape=(9,)))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dropout', 0, 0.5, step=0.1)))
    
    for i in range(hp.Int('num_layers', 1, 4)):
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=32, max_value=512, step=32), activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout_' + str(i), 0, 0.5, step=0.1)))
    
    model.add(Dense(len(label_encoder.classes_), activation='softmax'))
    
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


- ### Step  5: Initialize the Keras Tuner and Search for the Best Hyperparameters
  ##### We initialize the Keras Tuner and search for the best hyperparameters using the training and validation data.

In [16]:
# Initialize Keras Tuner
tuner = kt.Hyperband(build_model,
                     objective='val_accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='my_dir',
                     project_name='emotion_prediction')

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[keras.callbacks.EarlyStopping(patience=5)])


Trial 30 Complete [00h 00m 06s]
val_accuracy: 0.0

Best val_accuracy So Far: 0.043478261679410934
Total elapsed time: 00h 02m 16s


- ### Step 6: Retrieve the Best Model and Train it
  ##### We retrieve the best model found by the Keras Tuner and train it using the training data.

In [17]:
# Retrieve the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Train the best model
history = best_model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), callbacks=[keras.callbacks.EarlyStopping(patience=5)])


Epoch 1/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.0151 - loss: 4.7875 - val_accuracy: 0.0217 - val_loss: 5.0673
Epoch 2/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.1347 - loss: 4.0615 - val_accuracy: 0.0000e+00 - val_loss: 5.0844
Epoch 3/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.2043 - loss: 3.6472 - val_accuracy: 0.0000e+00 - val_loss: 5.1034
Epoch 4/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2109 - loss: 3.3158 - val_accuracy: 0.0000e+00 - val_loss: 5.1272
Epoch 5/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.2631 - loss: 2.9359 - val_accuracy: 0.0000e+00 - val_loss: 5.1588
Epoch 6/50
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.3600 - loss: 2.5519 - val_accuracy: 0.0000e+00 - val_loss: 5.1870


- ### Step 7: Evaluate the Model
  ##### We evaluate the trained model on the test data and print the test accuracy.

In [18]:
# Function to predict emotional word based on new RGB values
def predict_emotion(rgb1, rgb2, rgb3):
    rgb_values = np.hstack([rgb1, rgb2, rgb3]).reshape(1, -1)
    rgb_values = scaler.transform(rgb_values)
    prediction = best_model.predict(rgb_values)
    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

# Example prediction
new_rgb1 = [251, 167, 157]
new_rgb2 = [255, 242, 124]
new_rgb3 = [179, 22, 61]
print(predict_emotion(new_rgb1, new_rgb2, new_rgb3))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step
Cute


- ### Step 8: Save and Load the Model
  ##### Save the trained model to a file for future use, and show how to load it back when needed.

 - ### Step 9: Make Predictions
   #####  Define a function to predict the emotional word based on new RGB values by processing the input and using the trained model.